In [1]:
import pandas as pd
import re

In [2]:
df=pd.read_csv(r"E:\Projects\Machine Failures\Data\train.csv")
X_test=pd.read_csv(r"E:\Projects\Machine Failures\Data\test.csv")

In [89]:
df.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [4]:
df['Type'].unique()

array(['L', 'M', 'H'], dtype=object)

In [90]:
df.dtypes

id                           int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Tool wear [min]              int64
Machine failure              int64
TWF                          int64
HDF                          int64
PWF                          int64
OSF                          int64
RNF                          int64
dtype: object

In [91]:
def replace_spaces_with_underscore(column_labels):
    return [label.replace(' ', '_') for label in column_labels]

df.columns=replace_spaces_with_underscore(df.columns)

X_test.columns=replace_spaces_with_underscore(X_test)

In [92]:
def remove_special_characters(feature_names):
    # Define the regular expression pattern to match special characters
    pattern = r'[^\w\s]'
    
    # Initialize a list to store the cleaned feature names
    cleaned_feature_names = []
    
    # Iterate through each feature name and remove special characters
    for name in feature_names:
        cleaned_name = re.sub(pattern, '', name)
        cleaned_feature_names.append(cleaned_name)
    
    return cleaned_feature_names

df.columns=remove_special_characters(df.columns)
X_test.columns=remove_special_characters(X_test.columns)

In [93]:
df.head()

,id,Product_ID,Type,Air_temperature_K,Process_temperature_K,Rotational_speed_rpm,Torque_Nm,Tool_wear_min,Machine_failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [94]:
target_col = 'Machine_failure'

num_cols = [
    'Air_temperature_K',
    'Process_temperature_K',
    'Rotational_speed_rpm',
    'Torque_Nm',
    'Tool_wear_min'
]
binary_cols = [
    'TWF',
    'HDF',
    'PWF',
    'OSF',
    'RNF'
]

cat_cols=['Product_ID', 'Type']

In [96]:
def create_features(df):
    
    # Create a new feature by divided 'Air temperature' from 'Process temperature'
    df["Temperature_ratio"] = df['Process_temperature_K'] / df['Air_temperature_K']
    
    # Create a new feature by multiplying 'Torque' and 'Rotational speed'
    df['Torque * Rotational_speed'] = df['Torque_Nm'] * df['Rotational_speed_rpm']

    # Create a new feature by multiplying 'Torque' by 'Tool wear'
    df['Torque * Tool wear'] = df['Torque_Nm'] * df['Tool_wear_min']

    # Create a new feature by multiplying 'Torque' by 'Rotational speed'
    df['Torque * Rotational_speed'] = df['Torque_Nm'] * df['Rotational_speed_rpm']
    
    return df

In [99]:
df.dtypes

id                             int64
Product_ID                    object
Type                          object
Air_temperature_K            float64
Process_temperature_K        float64
Rotational_speed_rpm           int64
Torque_Nm                    float64
Tool_wear_min                  int64
Machine_failure                int64
TWF                           object
HDF                           object
PWF                           object
OSF                           object
RNF                           object
Temperature_ratio            float64
Torque * Rotational_speed    float64
Torque * Tool wear           float64
dtype: object

In [100]:
df=create_features(df=df)
X_test=create_features(df=X_test)

In [130]:
df[binary_cols] = df[binary_cols].astype(str)
X_test[binary_cols] = X_test[binary_cols].astype(str)

KeyError: "None of [Index(['TWF', 'HDF', 'PWF', 'OSF', 'RNF'], dtype='object')] are in the [columns]"

In [101]:
X_train=df.drop(columns='Machine_failure',axis=1)
y_train=df['Machine_failure']

### Ordinal Encoder

In [102]:
from category_encoders import OrdinalEncoder

In [104]:
X_train.drop(columns=['id','Product_ID'],axis=1,inplace=True)
X_test.drop(columns=['id','Product_ID'],axis=1,inplace=True)

### One hot encoder 

In [105]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [106]:
class onehot_class:
    def __init__(self, drop_first=True):
        self.drop_first = drop_first
        self.encoder = OneHotEncoder(sparse=False, drop='first')

    def fit_transform(self, df, categorical_columns):
        df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=self.drop_first)
        # Convert boolean columns to integer (0 and 1)
        df_encoded = df_encoded.astype(int)
        return df_encoded

    def fit_transform_numpy(self, X, categorical_indices):
        X_encoded = self.encoder.fit_transform(X[:, categorical_indices])
        # Convert boolean values to integer (0 and 1)
        X_encoded = X_encoded.astype(int)
        return X_encoded

    def get_feature_names(self, categorical_columns):
        feature_names = pd.get_dummies(['dummy'], columns=categorical_columns, drop_first=self.drop_first).columns.tolist()
        return feature_names

In [107]:
binary_cols

['TWF', 'HDF', 'PWF', 'OSF', 'RNF']

In [108]:
onehot_columns=['TWF', 'HDF', 'PWF', 'OSF', 'RNF','Type']

In [109]:

encoder = onehot_class(drop_first=True)

# For DataFrame
X_train = encoder.fit_transform(X_train,categorical_columns=onehot_columns )
X_test = encoder.fit_transform(X_test,categorical_columns=onehot_columns )



In [110]:
X_train

,Air_temperature_K,Process_temperature_K,Rotational_speed_rpm,Torque_Nm,Tool_wear_min,Temperature_ratio,Torque * Rotational_speed,Torque * Tool wear,TWF_1,HDF_1,PWF_1,OSF_1,RNF_1,Type_L,Type_M
0,300,309,1596,36,140,1,57615,5054,0,0,0,0,0,1,0
1,302,312,1759,29,200,1,51186,5820,0,0,0,0,0,0,1
2,299,308,1805,26,25,1,47832,662,0,0,0,0,0,1,0
3,301,310,1524,44,197,1,67513,8727,0,0,0,0,0,1,0
4,298,309,1641,35,34,1,58091,1203,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,300,311,1530,37,210,1,57375,7875,0,0,0,0,0,0,1
136425,297,308,1447,49,2,1,71047,98,0,0,0,0,0,0,0
136426,300,311,1524,38,214,1,58674,8239,0,0,0,0,0,1,0
136427,301,310,1447,46,42,1,66996,1944,0,0,0,0,0,1,0


### Scaling

In [111]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
import numpy as np


In [112]:
scaling_columns=['Air_temperature_K', 'Process_temperature_K',
       'Rotational_speed_rpm', 'Torque_Nm', 'Tool_wear_min', 'Temperature_ratio', 'Torque * Rotational_speed',
       'Torque * Tool wear']

In [113]:
X_train_numerical=X_train[scaling_columns]
X_test_numerical=X_test[scaling_columns]

In [114]:
X_train_categorical = X_train[X_train.columns.difference(scaling_columns)]
X_test_categorical = X_test[X_train.columns.difference(scaling_columns)]


In [115]:
# Create a custom function to apply the log transform
def log_transform(X):
    return np.log1p(X)  # Using np.log1p to avoid issues with zero and negative values

# Define the numerical_transformer with the log transform
numerical_transformer = Pipeline(steps=[
    ('log_transform', FunctionTransformer(log_transform, validate=False))
])

# Combine the numerical_transformer with the rest of the preprocessing steps
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, scaling_columns),
], remainder='passthrough')




In [116]:
'''
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, scaling_columns),
], remainder='passthrough')

'''


X_train_numerical_scaled = preprocessor.fit_transform(X_train_numerical)
X_test_numerical_scaled= preprocessor.fit_transform(X_test_numerical)


X_train_numerical_scaled = pd.DataFrame(X_train_numerical_scaled, columns=scaling_columns + list(X_train_numerical.columns[len(scaling_columns):]))
X_test_numerical_scaled = pd.DataFrame(X_test_numerical_scaled, columns=scaling_columns + list(X_test_numerical.columns[len(scaling_columns):]))


In [117]:
X_train_scaled = pd.concat([X_train_categorical, X_train_numerical_scaled], axis=1)
X_test_scaled = pd.concat([X_test_categorical, X_test_numerical_scaled], axis=1)

In [118]:
X_train_scaled

,HDF_1,OSF_1,PWF_1,RNF_1,TWF_1,Type_L,Type_M,Air_temperature_K,Process_temperature_K,Rotational_speed_rpm,Torque_Nm,Tool_wear_min,Temperature_ratio,Torque * Rotational_speed,Torque * Tool wear
0,0,0,0,0,0,1,0,5.707110,5.736572,7.375882,3.610918,4.948760,0.693147,10.961556,8.528133
1,0,0,0,0,0,0,1,5.713733,5.746203,7.473069,3.401197,5.303305,0.693147,10.843241,8.669227
2,0,0,0,0,0,1,0,5.703782,5.733341,7.498870,3.295837,3.258097,0.693147,10.775471,6.496775
3,0,0,0,0,0,1,0,5.710427,5.739793,7.329750,3.806662,5.288267,0.693147,11.120090,9.074292
4,0,0,0,0,0,0,1,5.700444,5.736572,7.403670,3.583519,3.555348,0.693147,10.969783,7.093405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,0,0,0,0,0,0,1,5.707110,5.743003,7.333676,3.637586,5.351858,0.693147,10.957381,8.971575
136425,0,0,0,0,0,0,0,5.697093,5.733341,7.277939,3.912023,1.098612,0.693147,11.171111,4.595120
136426,0,0,0,0,0,1,0,5.707110,5.743003,7.329750,3.663562,5.370638,0.693147,10.979769,9.016756
136427,0,0,0,0,0,1,0,5.710427,5.739793,7.277939,3.850148,3.761200,0.693147,11.112403,7.573017


In [119]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
import pandas as pd

### Splitter

In [120]:
class Splitter:
    def __init__(self, kfold=True, n_splits=5, cat_df=pd.DataFrame(), test_size=0.3):
        self.n_splits = n_splits
        self.kfold = kfold
        self.cat_df = cat_df
        self.test_size = test_size

    def split_data(self, X, y, random_state_list=[42]):

        for random_state in random_state_list:
            kf = StratifiedKFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
            for train_index, val_index in kf.split(X, self.cat_df):
                X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                yield X_train, X_val, y_train, y_val, val_index                        

### Model Classifier 

In [121]:

# Create an instance of Splitter for k-fold cross-validation with 5 folds
splitter = Splitter(kfold=True, n_splits=10, cat_df=y_train)

In [122]:
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


class ModelClassifier:
    def __init__(self):
        self.model_dict = {
            'LightGBM': lgb.LGBMClassifier(verbose=0),
            'CatBoost': CatBoostClassifier(silent=True, iterations=50),
            'XgBoost': xgb.XGBClassifier()
        }
        self.params = {
                        'LightGBM': {
                            'num_leaves': (10, 60),
                            'learning_rate': (0.01, 0.5),
                            'min_data_in_leaf': (10, 50),
                            'feature_fraction': (0.1, 0.9), 
                            'bagging_fraction': (0.5, 1.0)   
                        },
                        'CatBoost': {
                            'depth': (6, 16),
                            'learning_rate': (0.0, 0.5),
                            'l2_leaf_reg': (1, 10),
                            'iterations': (30, 30)
            
                        },
                        'XgBoost': {
                            'max_depth': (3, 50),
                            'learning_rate': (0.01, 0.4),
                            'n_estimators': (100, 500),
                            'min_child_weight': (1, 10)   
                        }
                    }   

    def fit_with_params(self, model_name, X, y):
        model = self.model_dict.get(model_name)
        if model:
            params = self.params.get(model_name, None)
            if params:
                print(f"Fitting {model_name} with specified parameters...")
                model.set_params(**params)
            else:
                print(f"Fitting {model_name} with default parameters.")
                # Implement fitting logic with default parameters

            return model.fit(X, y)
        else:
            raise ValueError(f"Model {model_name} not found. Please set the model first.")

    def fit_without_params(self, model_name, X, y):
        model = self.model_dict.get(model_name)
        if model:
            print(f"Fitting {model_name} without parameters.")
            return model.fit(X, y)
        else:
            raise ValueError(f"Model {model_name} not found. Please set the model first.")

### Ensemble

In [123]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class VotingEnsemble(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators, voting='hard'):
        self.estimators = estimators
        self.voting = voting

    def fit(self, X, y):
        for _, model in self.estimators:
            model.fit(X, y)
        return self

    def predict(self, X):
        if self.voting == 'hard':
            predictions = np.asarray([model.predict(X) for _, model in self.estimators])
            return np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=predictions)
        elif self.voting == 'soft':
            probabilities = np.asarray([model.predict_proba(X) for _, model in self.estimators])
            return np.argmax(np.mean(probabilities, axis=0), axis=1)
        else:
            raise ValueError(f"Invalid voting method: {self.voting}. Use 'hard' or 'soft'.")




## Optuna

In [124]:
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

class OptunaTuner_Catboost:
    def __init__(self, model, params, X, y):
        self.model = model
        self.params = params
        self.X = X
        self.y = y

    def Objective(self, trial):
        param_values = {}
        for key, value_range in self.params.items():
            if isinstance(value_range, tuple):  # Check if value_range is a tuple
                if value_range[0] <= value_range[1]:
                    if isinstance(value_range[0], int) and isinstance(value_range[1], int):
                        param_values[key] = trial.suggest_int(key, value_range[0], value_range[1])
                    else:
                        param_values[key] = trial.suggest_float(key, value_range[0], value_range[1])
                else:
                    raise ValueError(f"Invalid range for {key}: low={value_range[0]}, high={value_range[1]}")
            else:  # If value_range is not a tuple, treat it as a single value
                param_values[key] = value_range

        model = CatBoostClassifier(**param_values)
        
        # Initialize an empty list to store AUC-ROC scores from different folds
        auc_scores = []

        # Use the external data splitter for data splitting
        for X_Train, X_val, y_Train, y_val, val_index in tqdm(splitter.split_data(X=self.X, y=self.y)):
            # Perform tuning for each fold
            model.fit(X_Train, y_Train)  # Fit the model on the training data of this fold

            # Evaluate the model using AUC-ROC on the validation data of this fold
            y_probs = model.predict_proba(X_val)[:, 1]  # Get predicted probabilities for the positive class
            auc_roc = roc_auc_score(y_val, y_probs)
            
            
            # Append the AUC-ROC score of this fold to the list
            auc_scores.append(auc_roc)

        # Compute the average AUC-ROC score across all folds
        avg_auc_score = sum(auc_scores) / len(auc_scores)

        return avg_auc_score




    def tune(self, n_trials=100):
        study = optuna.create_study(direction="maximize")  # maximize AUC-ROC

        # Perform Optuna tuning
        study.optimize(self.Objective, n_trials=n_trials)

        best_params = study.best_params
        print(f"Best parameters: {best_params}")

        # Create a new CatBoost model instance with the best parameters
        best_model = CatBoostClassifier(**best_params)

        # Train the best model on the whole dataset
        best_model.fit(self.X, self.y)

        best_auc_score = study.best_value
        print(f"Best AUC Score: {best_auc_score}")

        # Here, we return both the tuned model and the best AUC-ROC score
        return best_auc_score, best_model


In [125]:
import optuna
from sklearn.metrics import roc_auc_score

class OptunaTuner:
    def __init__(self, model, params, X, y, splitter=splitter):
        self.model = model
        self.params = params
        self.X = X
        self.y = y
        self.splitter = splitter  # Assuming a data splitter object is passed

    def Objective(self, trial):
        param_values = {}
        for key, value_range in self.params.items():
            if value_range[0] <= value_range[1]:
                if isinstance(value_range[0], int) and isinstance(value_range[1], int):
                    param_values[key] = trial.suggest_int(key, value_range[0], value_range[1])
                else:
                    param_values[key] = trial.suggest_float(key, value_range[0], value_range[1])
            else:
                raise ValueError(f"Invalid range for {key}: low={value_range[0]}, high={value_range[1]}")

        self.model.set_params(**param_values)

        # Initialize an empty list to store AUC-ROC scores from different folds
        auc_scores = []

        for X_Train, X_val, y_Train, y_val, val_index in tqdm(self.splitter.split_data(X=self.X, y=self.y)):
            # Fit the model on the training data of this fold
            self.model.fit(X_Train, y_Train)

            # Evaluate the model using AUC-ROC on the validation data of this fold
            y_probs = self.model.predict_proba(X_val)[:, 1]
            auc_roc = roc_auc_score(y_val, y_probs)

            # Append the AUC-ROC score of this fold to the list
            auc_scores.append(auc_roc)

        # Compute the average AUC-ROC score across all folds
        avg_auc_score = sum(auc_scores) / len(auc_scores)

        return avg_auc_score

    def tune(self, n_trials=100):
        study = optuna.create_study(direction="maximize")  # maximize average AUC-ROC score
        study.optimize(self.Objective, n_trials=n_trials)

        best_params = study.best_params
        print(f"Best parameters: {best_params}")

        # Set the best parameters to the model
        self.model.set_params(**best_params)

        print(f"Best AUC Score: {study.best_value}")

        # Here, we return both the tuned model and the best AUC-ROC score
        return study.best_value, self.model

In [126]:
models = ModelClassifier()

results = {}  # Dictionary to store the best models and their AUC scores

# List to store the tuned models
tuned_models = []

for model_name, model in models.model_dict.items():
    if model_name == 'CatBoost':

        print(f"Tuning and fitting model ----------->>>>  {model_name}")

        # Create an instance of OptunaTuner for each model
        tuner = OptunaTuner_Catboost(model, params=models.params[model_name], X=X_train_scaled, y=y_train)

        # Perform hyperparameter tuning
        best_auc_score, tuned_model = tuner.tune(n_trials=5)

        print(f"Best AUC score for {model_name}: {best_auc_score}")
        print("----------------------")

        # Append the tuned model to the list of tuned models
        tuned_models.append((model_name, tuned_model))
        
        results[model_name] = best_auc_score
        
        
    else: 
        
        print(f"Tuning and fitting model ----------->>>>  {model_name}")

        # Create an instance of OptunaTuner for each model
        tuner = OptunaTuner(model, params=models.params[model_name], X=X_train_scaled, y=y_train)

        # Perform hyperparameter tuning
        best_auc_score, tuned_model = tuner.tune(n_trials=5)

        print(f"Best AUC score for {model_name}: {best_auc_score}")
        print("----------------------")

        # Append the tuned model to the list of tuned models
        tuned_models.append((model_name, tuned_model))
        
        results[model_name] = best_auc_score
        
# Convert the 'results' dictionary to a DataFrame
result_df = pd.DataFrame(results.items(), columns=['Model', 'Best AUC Score'])


[I 2023-08-05 20:03:05,365] A new study created in memory with name: no-name-a7123ec2-1e2f-4b55-b10b-f74bd0c662ed


Tuning and fitting model ----------->>>>  LightGBM


0it [00:00, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] feature_fraction is set=0.6127734679265118, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6127734679265118
[LightGBM] [Warning] bagging_fraction is set=0.6799202508651867, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6799202508651867
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=49, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=49
[LightGBM] [Warning] feature_fraction is set=0.6127734679265118, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6127734679265118
[LightGBM] [Warning] bagging_fraction is set=0.6799202508651867, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6799202508651867
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead 

1it [00:00,  2.69it/s]

...

Best parameters: {'depth': 6, 'learning_rate': 0.33358132413312563, 'l2_leaf_reg': 8, 'iterations': 30}
0:	learn: 0.4211259	total: 9.6ms	remaining: 278ms
1:	learn: 0.2795260	total: 18ms	remaining: 252ms
2:	learn: 0.1932839	total: 26.5ms	remaining: 238ms
3:	learn: 0.1378818	total: 34.8ms	remaining: 226ms
4:	learn: 0.1010840	total: 43.3ms	remaining: 217ms
5:	learn: 0.0762193	total: 51.9ms	remaining: 208ms
6:	learn: 0.0590168	total: 59.7ms	remaining: 196ms
7:	learn: 0.0472102	total: 68.3ms	remaining: 188ms
8:	learn: 0.0391494	total: 76.5ms	remaining: 179ms
9:	learn: 0.0337422	total: 84ms	remaining: 168ms
10:	learn: 0.0299912	total: 92.1ms	remaining: 159ms
11:	learn: 0.0273329	total: 101ms	remaining: 151ms
12:	learn: 0.0253109	total: 110ms	remaining: 144ms
13:	learn: 0.0243403	total: 118ms	remaining: 135ms
14:	learn: 0.0236229	total: 126ms	remaining: 126ms
15:	learn: 0.0227827	total: 135ms	remaining: 118ms
16:	learn: 0.0225575	total: 144ms	remaining: 110ms


[I 2023-08-05 20:06:20,627] A new study created in memory with name: no-name-a5f0be8d-6054-49cf-8361-ada725d7033f


17:	learn: 0.0222535	total: 152ms	remaining: 101ms
18:	learn: 0.0221281	total: 161ms	remaining: 93.1ms
19:	learn: 0.0218357	total: 170ms	remaining: 84.8ms
20:	learn: 0.0217161	total: 179ms	remaining: 76.7ms
21:	learn: 0.0215845	total: 188ms	remaining: 68.4ms
22:	learn: 0.0213308	total: 197ms	remaining: 59.9ms
23:	learn: 0.0212714	total: 205ms	remaining: 51.3ms
24:	learn: 0.0211177	total: 215ms	remaining: 43ms
25:	learn: 0.0210759	total: 222ms	remaining: 34.2ms
26:	learn: 0.0210520	total: 231ms	remaining: 25.7ms
27:	learn: 0.0208975	total: 240ms	remaining: 17.1ms
28:	learn: 0.0208739	total: 248ms	remaining: 8.56ms
29:	learn: 0.0208406	total: 257ms	remaining: 0us
Best AUC Score: 0.958952144205416
Best AUC score for CatBoost: 0.958952144205416
----------------------
Tuning and fitting model ----------->>>>  XgBoost


10it [05:19, 31.96s/it]
[I 2023-08-05 20:11:40,212] Trial 0 finished with value: 0.9612078998273972 and parameters: {'max_depth': 32, 'learning_rate': 0.03901587862264438, 'n_estimators': 311, 'min_child_weight': 7}. Best is trial 0 with value: 0.9612078998273972.
10it [08:59, 53.98s/it]
[I 2023-08-05 20:20:40,005] Trial 1 finished with value: 0.9554497378384511 and parameters: {'max_depth': 32, 'learning_rate': 0.20187463659787716, 'n_estimators': 455, 'min_child_weight': 6}. Best is trial 0 with value: 0.9612078998273972.
10it [04:33, 27.38s/it]
[I 2023-08-05 20:25:13,857] Trial 2 finished with value: 0.9526633201648573 and parameters: {'max_depth': 46, 'learning_rate': 0.2904515979308352, 'n_estimators': 161, 'min_child_weight': 1}. Best is trial 0 with value: 0.9612078998273972.
10it [03:23, 20.36s/it]
[I 2023-08-05 20:28:37,467] Trial 3 finished with value: 0.9520068963635773 and parameters: {'max_depth': 43, 'learning_rate': 0.2858533812028478, 'n_estimators': 124, 'min_child_wei

Best parameters: {'max_depth': 32, 'learning_rate': 0.03901587862264438, 'n_estimators': 311, 'min_child_weight': 7}
Best AUC Score: 0.9612078998273972
Best AUC score for XgBoost: 0.9612078998273972
----------------------


In [131]:
result_df

,Model,Best AUC Score
0,LightGBM,0.963902
1,CatBoost,0.958952
2,XgBoost,0.961208


In [132]:
def make_predictions(models_list, data):
    """
    Function to make predictions using a list of tuned models and store the results in a DataFrame.

    Parameters:
        models_list (list): List of tuples containing (model_name, model_object).
        data (pd.DataFrame): Input data on which to make predictions.

    Returns:
        pd.DataFrame: DataFrame with model predictions as columns.
    """
    predictions_df = pd.DataFrame()
    
    for model_name, model_object in models_list:
        predictions = model_object.predict(data)  # Assuming your model has a predict() method
        predictions_df[model_name] = predictions
    
    return predictions_df

In [133]:
predictions_df=make_predictions(models_list=tuned_models,data=X_test_scaled)

[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=38
[LightGBM] [Warning] feature_fraction is set=0.6915671964702581, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6915671964702581
[LightGBM] [Warning] bagging_fraction is set=0.7956871806957572, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7956871806957572


In [134]:
predictions_df.head()

,LightGBM,CatBoost,XgBoost
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [135]:
y_pred=predictions_df['CatBoost'].values

In [136]:
dataframe=pd.DataFrame()
prediction_test=pd.read_csv(r"E:\Projects\Machine Failures\Data\test.csv")
dataframe['id']=prediction_test['id']
dataframe['Machine failure']=y_pred
dataframe.to_csv('submission.csv',index=False)

## Ensemble

In [137]:
# Create a list of (name, model) tuples for the voting ensemble
estimators = tuned_models

# Create a VotingEnsemble with soft voting
voting_ensemble = VotingEnsemble(estimators, voting='hard')

# Fit and predict using the voting ensemble
voting_ensemble.fit(X_train_scaled, y_train)
y_pred = voting_ensemble.predict(X_test_scaled)


[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=38
[LightGBM] [Warning] feature_fraction is set=0.6915671964702581, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6915671964702581
[LightGBM] [Warning] bagging_fraction is set=0.7956871806957572, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7956871806957572
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=38, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=38
[LightGBM] [Warning] feature_fraction is set=0.6915671964702581, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6915671964702581
[LightGBM] [Warning] bagging_fraction is set=0.7956871806957572, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7956871806957572
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead 

In [138]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [139]:
dataframe=pd.DataFrame()
prediction_test=pd.read_csv(r"E:\Projects\Machine Failures\Data\test.csv")
dataframe['id']=prediction_test['id']
dataframe['Machine failure']=y_pred
dataframe.to_csv('submission.csv',index=False)